In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split 

df = pd.read_csv('../data/data.csv')
print(df.shape)
df = df.dropna(subset=['state']) #罪过 最后要悄悄删掉
df = df.dropna(subset=['price'])
df=df.drop(columns=["zip_code"])
print(df.shape)
y = df['price']
X = df.loc[:, df.columns != 'price']
# Counting occurrences of each state
state_counts = X['state'].value_counts()

# Set common and rare states
common_states = state_counts[state_counts >= 2].index
rare_states = state_counts[state_counts < 2].index

# splitting the dataset
mask_common = X['state'].isin(common_states)
mask_rare = X['state'].isin(rare_states)

X_common, y_common = X[mask_common], y[mask_common]
X_rare, y_rare = X[mask_rare], y[mask_rare]

random_state = 42

X_train, X_other, y_train, y_other = train_test_split(
    X_common, y_common, train_size=0.98, stratify=X_common['state'], random_state=random_state
)

X_val, X_test, y_val, y_test = train_test_split(
    X_other, y_other, train_size=0.5, random_state=random_state
)

X_train = pd.concat([X_train, X_rare])
y_train = pd.concat([y_train, y_rare])


print("Train size:", len(X_train))
print("Validation size:", len(X_val))
print("Test size:", len(X_test))


(2226382, 12)
(2224833, 11)
Train size: 2180336
Validation size: 22248
Test size: 22249


In [54]:
# Check NA for each column

cols = ['brokered_by', 'status', 'price', 'bed', 'bath', 
        'acre_lot', 'street', 'city', 'state', 'zip_code', 
        'house_size', 'prev_sold_date']

na_summary = pd.DataFrame({
    'Column': cols,
    'Data_Type': [df[c].dtype for c in cols],
    'Num_NA': [df[c].isna().sum() for c in cols],
    'NA_Rate': [round(df[c].isna().mean(), 3) for c in cols]
})

na_summary = na_summary.sort_values('Num_NA', ascending=False).reset_index(drop=True)

print(na_summary)


            Column Data_Type  Num_NA  NA_Rate
0   prev_sold_date    object  733248    0.330
1       house_size   float64  567872    0.255
2             bath   float64  510984    0.230
3              bed   float64  480859    0.216
4         acre_lot   float64  325134    0.146
5           street   float64   10864    0.005
6      brokered_by   float64    4533    0.002
7             city    object    1404    0.001
8         zip_code   float64     296    0.000
9           status    object       0    0.000
10           price   float64       0    0.000
11           state    object       0    0.000


In [55]:
# Check outliers for numeric columns using IQR method

def detect_outliers_iqr(df, columns):
    outlier_summary = []
    for col in columns:
        if pd.api.types.is_numeric_dtype(df[col]):  # Check if the column is numeric
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            lower = Q1 - 1.5 * IQR
            upper = Q3 + 1.5 * IQR
            outliers = ((df[col] < lower) | (df[col] > upper)).sum()
            outlier_ratio = round(outliers / len(df), 3)
            col_min = df[col].min()
            col_max = df[col].max()
            
            outlier_summary.append([
                col,
                df[col].dtype,
                round(df[col].min(), 2),
                round(df[col].max(), 2),
                round(Q1, 2),
                round(Q3, 2),
                round(lower, 2),
                round(upper, 2),
                outliers,
                outlier_ratio
            ])
    
    outlier_table = pd.DataFrame(outlier_summary, columns=[
        'Column', 'Data_Type', 'Min', 'Max', 'Q1', 'Q3', 
        'Lower_Bound', 'Upper_Bound', 'Num_Outliers', 'Outlier_Rate'
    ])
    
    return outlier_table.sort_values('Outlier_Rate', ascending=False).reset_index(drop=True)

numeric_cols = ['price', 'bed', 'bath', 'acre_lot', 'house_size']
outlier_table = detect_outliers_iqr(df, numeric_cols)
print(outlier_table)


       Column Data_Type  Min           Max         Q1         Q3  Lower_Bound  \
0    acre_lot   float64  0.0  1.000000e+05       0.15       0.98         -1.1   
1       price   float64  0.0  2.147484e+09  165000.00  550000.00    -412500.0   
2         bed   float64  1.0  4.730000e+02       3.00       4.00          1.5   
3        bath   float64  1.0  8.300000e+02       2.00       3.00          0.5   
4  house_size   float64  4.0  1.040400e+09    1300.00    2413.00       -369.5   

   Upper_Bound  Num_Outliers  Outlier_Rate  
0         2.22        292018         0.131  
1   1127500.00        171600         0.077  
2         5.50        118888         0.053  
3         4.50         79063         0.036  
4      4082.50         77831         0.035  


In [ ]:
# Preprocessing Pipeline

# preprocess with pipeline and columntransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.base import BaseEstimator, TransformerMixin
from datetime import datetime



# collect the various features
cat_ftrs = ['status','city','state']
num_ftrs = ['bed','bath','acre_lot','house_size']
zip_ftr = 'zip_code'
broker_ftr = 'brokered_by'
street_ftr = 'street'
sold_ftr = 'prev_sold_date'


random_state = 42

class BinaryTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        # NA → 0；非 NA → 1
        return np.where(pd.isna(X), 0, 1).reshape(-1,1)

class ZipCodeTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        if isinstance(X, pd.DataFrame):
            X = X.iloc[:, 0]
        zip_str = X.astype(str)
        zip_str = zip_str.str.split('.').str[0]
        # 去掉非数字字符（例如 "#####", "nan", etc.）
        zip_str = zip_str.apply(lambda s: ''.join(ch for ch in s if ch.isdigit()))
        # 空字符串 → 视为缺失
        zip_str = zip_str.replace('', np.nan)
        # 缺失 ZIP → 用 "00000" 填充
        zip_str = zip_str.fillna("00000")
        # ⭐ 最重要：统一补足为 5 位 ZIP（不足前补零）
        zip_str = zip_str.str.zfill(5)
        return zip_str.values.reshape(-1, 1)

    
class PrevSoldDateTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        if isinstance(X, pd.DataFrame):
            X = X.iloc[:, 0]     # 只有一列
        else:
            X = pd.Series(X)

        dates = pd.to_datetime(X, errors='coerce')
        years = (datetime.now() - dates).dt.days / 365
        years = years.fillna(-1)

        return years.values.reshape(-1,1)



# one-hot encoder and imputer
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant',fill_value='missing')),
    ('onehot', OneHotEncoder(sparse_output=True,handle_unknown='ignore'))])
# standard scaler and imputer
numeric_transformer = Pipeline(steps=[
    ('imputer', IterativeImputer(max_iter=10, random_state=42)),
    ('scaler', StandardScaler())])

tree_numeric_transformer = Pipeline(steps=[
    ('imputer', IterativeImputer(max_iter=10, random_state=42))])

broker_pipe = Pipeline(steps=[
    ('binary', BinaryTransformer())
])
street_pipe = Pipeline(steps=[
    ('binary', BinaryTransformer())
])
zip_pipe = Pipeline(steps=[
    ('zip_transform', ZipCodeTransformer()),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=True))
])
sold_pipe = Pipeline(steps=[
    ('sold_transform', PrevSoldDateTransformer()),
    ('scaler', StandardScaler())
])

# collect the transformers
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_ftrs),
        ('cat', categorical_transformer, cat_ftrs),
        ('broker', broker_pipe, broker_ftr),
        ('street', street_pipe, street_ftr),
        ('zip', zip_pipe, zip_ftr),
        ('sold', sold_pipe, sold_ftr)
    ],
    remainder='drop'
)


tree_preprocessor = ColumnTransformer(
    transformers=[
        ('num', tree_numeric_transformer, num_ftrs),
        ('cat', categorical_transformer, cat_ftrs),
        ('broker', broker_pipe, broker_ftr),
        ('street', street_pipe, street_ftr),
        ('zip', zip_pipe, zip_ftr),
        ('sold', sold_pipe, sold_ftr)
    ],
    remainder='drop'
)


xgb_preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, cat_ftrs),
        ('broker', broker_pipe, broker_ftr),
        ('street', street_pipe, street_ftr),
        ('zip', zip_pipe, zip_ftr),
        ('sold', sold_pipe, sold_ftr)
    ],
    remainder='drop'
)


In [8]:
# save small dataset
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split 

df = pd.read_csv('../data/data.csv')
df = df.dropna(subset=['state']) #罪过 最后要悄悄删掉
df = df.dropna(subset=['price'])
# df=df.drop(columns=["zip_code"])
print(df.shape)
X = df.loc[:, df.columns != 'price']
# Counting occurrences of each state
state_counts = X['state'].value_counts()
# Set common and rare states
common_states = state_counts[state_counts >= 2].index
rare_states = state_counts[state_counts < 2].index
# splitting the dataset
mask_common = X['state'].isin(common_states)
mask_rare = X['state'].isin(rare_states)
X_common, y_common = X[mask_common], y[mask_common]
X_rare, y_rare = X[mask_rare], y[mask_rare]


X_small_c, _, y_small_c, _ = train_test_split(
    X_common,
    y_common,
    test_size=0.99,                     # keep 1%
    stratify=X_common['state'],
    random_state=42
)

X_small = pd.concat([X_small_c, X_rare], axis=0).reset_index(drop=True)
y_small = pd.concat([y_small_c, y_rare], axis=0).reset_index(drop=True)

X_small.to_csv("../data/X_small.csv", index=False)
y_small.to_csv("../data/y_small.csv", index=False)


(2224833, 12)
